In [94]:
import re
import numpy as np

In [2]:
example = '''
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
'''

In [72]:
def parse(text):
    matrix = []
    for line in text.split('\n'):
        if(line):
            matrix.append(['.', *list(line), '.'])
    
    matrix.insert(0, ['.']*len(matrix[0]))
    matrix.append(['.']*len(matrix[0]))
    return np.array(matrix)
            
            
m = parse(example)
m

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '4', '6', '7', '.', '.', '1', '1', '4', '.', '.', '.'],
       ['.', '.', '.', '.', '*', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '3', '5', '.', '.', '6', '3', '3', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
       ['.', '6', '1', '7', '*', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '+', '.', '5', '8', '.', '.'],
       ['.', '.', '.', '5', '9', '2', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '7', '5', '5', '.', '.'],
       ['.', '.', '.', '.', '$', '.', '*', '.', '.', '.', '.', '.'],
       ['.', '.', '6', '6', '4', '.', '5', '9', '8', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U1')

In [102]:
def process(text):
    m = parse(text)

    l = 0
    c = 0

    c_start = 0
    c_end = 0
    number_found = False
    while l < len(m):
        if(m[l,c] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) and (not number_found):
            c_start = c-1
            number_found = True
        elif (m[l,c] not in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) and (number_found):
            c_end = c+1
            number_found = False
            # scan the area
            area_to_scan = m[l-1:l+2, c_start:c_end]
            if not len(np.where(~np.isin(area_to_scan, ['.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']))[0]):
                # delete numbers
                m[l, c_start+1:c_end-1] = '.'

        c += 1

        if c >= len(m[0]):
            c = 0
            l += 1

    m = m.tolist()
    m = [''.join(line) for line in m]
    m = ''.join(m)
    return sum([int(n) for n in re.findall(r'\d+', m)])

process(example)

4361

In [112]:
def process_2(text):    
    m = parse(text)
    
    gear_rows, gear_cols = np.where(m == '*')
    
    gears = [{'l':l, 'c':c, 'ns':[]} for l, c in zip(gear_rows, gear_cols)]

    l = 0
    c = 0

    c_start = 0
    c_end = 0
    number_found = False
    while l < len(m):
        if(m[l,c] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) and (not number_found):
            c_start = c-1
            number_found = True
        elif (m[l,c] not in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) and (number_found):
            c_end = c+1
            number_found = False
            # scan the area
            area_to_scan = m[l-1:l+2, c_start:c_end]
#             print(area_to_scan)
            if not len(np.where(~np.isin(area_to_scan, ['.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']))[0]):
                # delete numbers
                m[l, c_start+1:c_end-1] = '.'
            else:
                for gear in gears[:]:
                    if (gear['l'] >= l-1) and (gear['l'] <= l+1):
                        if (gear['c'] >= c_start) and (gear['c'] < c_end):
                            gear['ns'].append(int(''.join(m[l, c_start+1:c_end-1].tolist())))

        c += 1

        if c >= len(m[0]):
            c = 0
            l += 1

    m = m.tolist()
    m = [''.join(line) for line in m]
    m = ''.join(m)
    return sum([g['ns'][0]*g['ns'][1] for g in gears if len(g['ns']) == 2])

process_2(example)

467835

In [113]:
import os
with open('../token.txt') as f:
    os.environ["AOC_SESSION"] = f.readline()

import aocd
instructions = aocd.get_data(day=3, year=2023)
print(process(instructions))
print(process_2(instructions))

544433
76314915
